In [7]:
import os
import scipy.io as sio
import numpy as np
from itertools import product
from collections import namedtuple
from joblib import Parallel, delayed
import emg_features

In [8]:
subjects = list(range(0,27))  #27 person
#subjects=[0] # for test only person 0
gestures = list(range(1,53))  #52 different gestures
#gestures=[1] #for test only gesture 1
trials = list(range(10))      #10 channel

In [38]:
source_path='C:/Users/11488/Desktop/my semg/data/db-1'

### Parameter

In [10]:
filtering_type = 'lowpass'
framerate = 100

window_length_ms = 200
window_stride_ms = 10

window = int(window_length_ms*framerate/1000)   #20
stride = int(window_stride_ms*framerate/1000)   #1


In [11]:
output_path = ('C:/Users/11488/Desktop/my semg/wwt/ninapro-feature/ninapro-db1-var-raw-prepro-%s-win-%d-stride-%d' % (filtering_type, window, stride))

In [12]:
Combo = namedtuple('Combo', ['subject', 'gesture', 'trial']) #define a class Comb with attribute "subject" "gesture" "trial"

In [13]:
feature_list = ['ssc', 'var']  #slop sign change, variance

In [140]:
# tr=sio.loadmat('C:/Users/11488/Desktop/my semg/data/db-1/s1/S1_A1_E1.mat')
# tr.keys()
# fl=0
# s_symbol=[]
# e_symbol=[]
# tr['stimulus']
# for i in range(len(tr['stimulus'])):
#     if fl==1 and tr['stimulus'][i][0]==0:
#         e_symbol.append(i)
#         fl=0
        
#     if fl==0 and tr['stimulus'][i][0]!=0:
#         s_symbol.append(i)
#         fl=1
    
    
# print(s_symbol)    
# print(e_symbol)    
# for i in range(len(s_symbol)):
#     print(s_symbol[i]-e_symbol[i])

### Pre-Process Nina-database 1

In [99]:
for s in range(1,len(subjects)+1):
    s_path=os.path.join(source_path,'s'+str(s))
#     print('process data for subjects '+str(s)+':')
    for d in range(1,4):
        open_mat=os.path.join(s_path,'S'+str(s)+'_A1_E'+str(d))
#         print('open mat: '+ open_mat)
        my_mat = sio.loadmat(open_mat)
        for g in range(1,len(gestures)):
            out_dir=os.path.join(s_path,
                          str(s).rjust(3,'0'),
                          str(g).rjust(3,'0'))
            if os.path.isdir(out_dir) is False:
                os.makedirs(out_dir)
            for t in range(len(trials)):
                path=os.path.join(out_dir,
                      str(s).rjust(3,'0')+'_'+str(g).rjust(3,'0')+'_'+str(t).rjust(3,'0')+'.mat')
                ch=dict()
                da=[]
                if d==1:
                    if g>=13:
                        pass
                    else:
                        for j in range( my_mat['stimulus'].shape[0]):
                            if my_mat['stimulus'][j]==[g]:
                                da.append(my_mat['emg'][i][t])
                        ch['data']=da
                        sio.savemat(path, ch)  
                        
                elif d==2:
                    if g<13 or g>=30:
                        pass
                    else:
                        for j in range( my_mat['stimulus'].shape[0]):
                            if my_mat['stimulus'][j]==[g-12]:
                                da.append(my_mat['emg'][i][t])
                        ch['data']=da
                        sio.savemat(path, ch) 
                elif d==3:
                    if g<30:
                        pass
                    else:
                        for j in range( my_mat['stimulus'].shape[0]):
                            if my_mat['stimulus'][j]==[g-29]:
                                da.append(my_mat['emg'][i][t])
                        ch['data']=da
                        sio.savemat(path, ch) 
                        
                    

In [103]:
#process for gesture 0 rest
g=0
for s in range(1,len(subjects)+1):
    s_path=os.path.join(source_path,'s'+str(s))
#     print('process data for subjects '+str(s)+':')
    for d in range(1,4):
        open_mat=os.path.join(s_path,'S'+str(s)+'_A1_E'+str(d))
#         print('open mat: '+ open_mat)
        my_mat = sio.loadmat(open_mat)
        out_dir=os.path.join(s_path,
                     str(s).rjust(3,'0'),
                     str(g).rjust(3,'0'))
        if os.path.isdir(out_dir) is False:
            os.makedirs(out_dir)
        for t in range(len(trials)):
            path=os.path.join(out_dir,
                    str(s).rjust(3,'0')+'_'+str(g).rjust(3,'0')+'_'+str(t).rjust(3,'0')+'.mat')
            ch=dict()
            da=[]
            for j in range( my_mat['stimulus'].shape[0]):
                if my_mat['stimulus'][j]==[g]:
                      da.append(my_mat['emg'][i][t])
            ch['data']=da
            sio.savemat(path, ch)  
                

### Loand and Check

In [25]:
combos = get_combos(product(subjects, gestures, trials))        
combos = list(combos)
source_path='C:/Users/11488/Desktop/my semg/data/db-1'
# print(combos)
# data_set=[[]for i in range(len(trials))]
for combo in combos:
    in_path = os.path.join(
                    source_path,
                    
                    '{c.subject:03d}',
                    '{c.gesture:03d}',
                    '{c.subject:03d}_{c.gesture:03d}_{c.trial:03d}.mat').format(c=combo)
#    print(in_path)
    
    #print(in_path)
#     tr=sio.loadmat(in_path)
#     data_set[combo.trial]=tr['data']
#     print('tr'+str(combo.trial)+'shape:',tr['data'].shape )

### Define Function

In [15]:
def get_combos(*args):
    for arg in args:
        if isinstance(arg, tuple):
            arg = [arg]
        for a in arg:
            yield Combo(*a)
           

In [16]:
def butter_lowpass_filter(data, cut, fs, order, zero_phase=False):
    from scipy.signal import butter, lfilter, filtfilt

    nyq = 0.5 * fs
    cut = cut / nyq

    b, a = butter(order, cut, btype='low')
    y = (filtfilt if zero_phase else lfilter)(b, a, data)
    return y

In [17]:
def get_segments(data, window, stride):
    return windowed_view(
        data.flat,
        window * data.shape[1],
        (window-stride)* data.shape[1]
    )

def windowed_view(arr, window, overlap):
    from numpy.lib.stride_tricks import as_strided
    arr = np.asarray(arr)
    window_step = window - overlap
    new_shape = arr.shape[:-1] + ((arr.shape[-1] - overlap) // window_step,
                                  window)
    new_strides = (arr.strides[:-1] + (window_step * arr.strides[-1],) +
                   arr.strides[-1:])
    return as_strided(arr, shape=new_shape, strides=new_strides)

In [32]:
def emg_feature_extraction_parallel_2(input_path, output_path, combo, feature_list):
    
    in_path = os.path.join(
                input_path, 
                's'+'{c.subject}',
                '{c.subject:03d}',
                '{c.gesture:03d}',
                '{c.subject:03d}_{c.gesture:03d}_{c.trial:03d}.mat').format(c=combo)
                
    out_dir = os.path.join(
                output_path,
                '{c.subject:03d}',
                '{c.gesture:03d}').format(c=combo)  
                
    if os.path.isdir(out_dir) is False:
        
        os.makedirs(out_dir)                 
     
                
    data = sio.loadmat(in_path)['data'].astype(np.float32)
        
    print ("Subject %d Gesture %d Trial %d data loaded!" % (combo.subject, combo.gesture, combo.trial))
      
        
      
#     if filtering_type is 'lowpass':      
        
#             data = np.transpose([lowpass(ch, 1, framerate, 1, zero_phase=True) for ch in data.T])
    data = np.transpose([butter_lowpass_filter(ch, 1, framerate, 1, zero_phase=True) for ch in data.T])   
    print ("Subject %d Gesture %d Trial %d bandpass filtering finished!" % (combo.subject, combo.gesture, combo.trial))
#     else:
#         pass
           
# #      data = downsample(data, step=20)        
        
    chnum = data.shape[1];     
    data = get_segments(data, window, stride)
    data = data.reshape(-1, window, chnum)
      
    for feature_name in feature_list:
        feature = [np.transpose(extract_emg_feature(seg.T, feature_name)) for seg in data]
        feature = np.array(feature)
        out_path = os.path.join(
                                  out_dir,
                                  '{0.subject:03d}_{0.gesture:03d}_{0.trial:03d}_{1}.mat').format(combo, feature_name)  
        sio.savemat(out_path, {'data': feature, 'label': combo.gesture, 'subject': combo.subject, 'trial':combo.trial})                        
        print ("Subject %d Gesture %d Trial %d %s saved!" % (combo.subject, combo.gesture, combo.trial, feature_name))

### Process Begin here

In [33]:
Subjects = list(range(1,28))  #27 person
#subjects=[0] # for test only person 0
Gestures = list(range(0,52))  #52 different gestures
#gestures=[1] #for test only gesture 1
Trials = list(range(10))      #10 channel

In [56]:
combo=combos[107]
in_path = os.path.join(
                source_path, 
                's'+'{c.subject}',
                '{c.subject:03d}',
                '{c.gesture:03d}',
                '{c.subject:03d}_{c.gesture:03d}_{c.trial:03d}.mat').format(c=combo)
                
out_dir = os.path.join(
                output_path,
                '{c.subject:03d}',
                '{c.gesture:03d}').format(c=combo)  
                
if os.path.isdir(out_dir) is False:
        
    os.makedirs(out_dir)                 
     
                
data = sio.loadmat(in_path)['data'].astype(np.float32)

        
print ("Subject %d Gesture %d Trial %d data loaded!" % (combo.subject, combo.gesture, combo.trial))
# for i in data:
#     for j in i:
#         print(j)
data

Subject 1 Gesture 10 Trial 7 data loaded!


array([[0.0391, 0.0391, 0.0391, ..., 0.0391, 0.0391, 0.0391]],
      dtype=float32)

In [39]:

combos = get_combos(product(Subjects, Gestures, Trials))
           
combos = list(combos)

Parallel(n_jobs=8)(delayed(emg_feature_extraction_parallel_2)(source_path, output_path, combo, feature_list) for combo in combos)     
# in_path = os.path.join(
#                 source_path,
#                 '{c.subject:03d}',
#                 '{c.gesture:03d}',
#                 '{c.subject:03d}_{c.gesture:03d}_{c.trial:03d}.mat').format(c=combo)
# out_dir = os.path.join(
#                     output_path,
#                     '{c.subject:03d}',
#                     '{c.gesture:03d}').format(c=combo)  
                
# if os.path.isdir(out_dir) is False:
#     os.makedirs(out_dir)               
# data = sio.loadmat(in_path)['data'].astype(np.float32)
# print ("Subject %d Gesture %d Trial %d data loaded!" % (combo.subject, combo.gesture, combo.trial))

# if filtering_type == 'lowpass':            
# #             data = np.transpose([lowpass(ch, 1, framerate, 1, zero_phase=True) for ch in data.T])
#     data = np.transpose([butter_lowpass_filter(ch, 1, framerate, 1, zero_phase=True) for ch in data])   
#     print ("Subject %d Gesture %d Trial %d bandpass filtering finished!" % (combo.subject, combo.gesture, combo.trial))
# else:
#     pass
# chnum = data.shape[1] 

# data = get_segments(data, window, stride)
# data = data.reshape(-1, window, chnum)


ValueError: The length of the input vector x must be greater than padlen, which is 6.

In [253]:
in_path

'C:/Users/11488/Desktop/my semg/data\\data\\000\\001\\000_001_000.mat'

In [ ]:
def emg_feature_extraction_parallel_2(input_path, output_path, combo, feature_list):
    in_path = os.path.join(
                input_path, 'data',
                '{c.subject:03d}',
                '{c.gesture:03d}',
                '{c.subject:03d}_{c.gesture:03d}_{c.trial:03d}.mat').format(c=combo)
                
    out_dir = os.path.join(
                    output_path,
                    '{c.subject:03d}',
                    '{c.gesture:03d}').format(c=combo)  
                
    if os.path.isdir(out_dir) is False:
             os.makedirs(out_dir)                 
     
                
    data = sio.loadmat(in_path)['data'].astype(np.float32)
        
    print ("Subject %d Gesture %d Trial %d data loaded!" % (combo.subject, combo.gesture, combo.trial))
      
        
      
    if filtering_type is 'lowpass':            
#             data = np.transpose([lowpass(ch, 1, framerate, 1, zero_phase=True) for ch in data.T])
        data = np.transpose([butter_lowpass_filter(ch, 1, framerate, 1, zero_phase=True) for ch in data.T])   
        print ("Subject %d Gesture %d Trial %d bandpass filtering finished!" % (combo.subject, combo.gesture, combo.trial))
    else:
        pass
           
#      data = downsample(data, step=20)        
        
    chnum = data.shape[1]    
    data = get_segments(data, window, stride)
      data = data.reshape(-1, window, chnum)
      
      for feature_name in feature_list:
          feature = [np.transpose(extract_emg_feature(seg.T, feature_name)) for seg in data]
          feature = np.array(feature)
          out_path = os.path.join(
                                  out_dir,
                                  '{0.subject:03d}_{0.gesture:03d}_{0.trial:03d}_{1}.mat').format(combo, feature_name)  
          sio.savemat(out_path, {'data': feature, 'label': combo.gesture, 'subject': combo.subject, 'trial':combo.trial})                        
          print ("Subject %d Gesture %d Trial %d %s saved!" % (combo.subject, combo.gesture, combo.trial, feature_name))


In [244]:
print ("NinaPro feature map generation, use window = %d frames, stride = %d frames" % (window, stride)) 

combos = get_combos(product(subjects, gestures, trials))
           
combos = list(combos)

# combos

NinaPro feature map generation, use window = 20 frames, stride = 1 frames


In [ ]:
Parallel(n_jobs=8)(delayed(emg_feature_extraction_parallel_2)(input_path, output_path, combo, feature_list) for combo in combos) 

In [11]:
def windowed_view(arr, window, overlap):
    from numpy.lib.stride_tricks import as_strided
    arr = np.asarray(arr)
    window_step = window - overlap
    new_shape = arr.shape[:-1] + ((arr.shape[-1] - overlap) // window_step,
                                  window)
    print(arr.shape[:-1])
    print(((arr.shape[-1] - overlap) // window_step,
                                  window))
    print(new_shape)
    new_strides = (arr.strides[:-1] + (window_step * arr.strides[-1],) +
                   arr.strides[-1:])
    return as_strided(arr, shape=new_shape, strides=new_strides)

In [23]:
arr=np.zeros((1,10000))
x=windowed_view(arr,1000,950)
x.reshape(-1, 20, 500)

(1,)
(181, 1000)
(1, 181, 1000)


ValueError: cannot reshape array of size 181000 into shape (20,500)